In [1]:
import sys
sys.path.append("..")

from molsberry.core import Pipeline, InputBlock, OutputBlock
from molsberry.core import SDFPathRep, PDBPathRep
from molsberry.core import LigandData, ProteinData

from molsberry.modules.mopac import MOPACMozymeConfig
from molsberry.modules.mopac import (
    MOPACSinglePointCalculator, MOPACPLInteractionCalculator,
    MOPACLigandOptimizer
)

In [2]:
class BindingEnergyCalcPipe(Pipeline):
    name = "bindenpipe"
    display_name = "Binding Energy Calculation Pipeline"
    def build(self):
        config = MOPACMozymeConfig()
        config.keywords.extend(["PM6-D3H4X", "EPS=78.5", "VDW(C=1.821;N=1.904;O=1.682;H=0.828)", "NOMM"])
        self.add_block(InputBlock(["ligand", "protein"]), "input")
        self.add_block(MOPACPLInteractionCalculator(config), "intercalc")
        self.add_block(MOPACLigandOptimizer(config, "LBFGS"), "ligopt")
        self.add_block(OutputBlock(["e_lig_init", "e_lig_opt", "e_interaction"]), "output")
        self.add_connection("input", "ligand", "intercalc", "ligands")
        self.add_connection("input", "protein", "intercalc", "proteins")
        self.add_connection("input", "ligand", "ligopt", "ligands")
        self.add_connection("intercalc", "e_interaction", "output", "e_interaction")
        self.add_connection("ligopt", "e_init", "output", "e_lig_init")
        self.add_connection("ligopt", "e_final", "output", "e_lig_opt")

In [3]:
ligdata = LigandData(SDFPathRep("lig.sdf"))
pocdata = ProteinData(PDBPathRep("poc.pdb"))

In [4]:
pipeline = BindingEnergyCalcPipe(base_dir="./example1_out")
out = pipeline.execute(ligand=ligdata, protein=pocdata)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>   STARTED: Binding Energy Calculation Pipeline    >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (intercalc) MOPAC Protein Ligand Interaction Calculator]
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f5304c67b90> <class 'molsberry.core.data.molecules.LigandData'>
<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>
<class 'molsberry.core.data.molecules.ProteinData'>
<molsberry.core.data.molecules.ProteinData object at 0x7f5304c84c50> <class 'molsberry.core.data.molecules.ProteinData'>
<class 'molsberry.modules.mopac.representations.MOPACInputMolRep'>

[Running Pipe Block: (ligopt) MOPAC Ligand Optimizer]
<class 'molsberry.core.data.molecules.LigandData'>
<molsberry.core.data.molecules.LigandData object at 0x7f5304c67b90> <class 'molsberry.core.data.molecules.LigandData'>
[<class 'molsberry.modules.mopac.representations.M

In [7]:
out.items()

dict_items([('e_interaction', <molsberry.core.data.collections.BatchedData object at 0x7f5304c98e90>), ('e_lig_init', <molsberry.core.data.collections.BatchedData object at 0x7f5304c9a410>), ('e_lig_opt', <molsberry.core.data.collections.BatchedData object at 0x7f5304c9a550>)])

In [8]:
out.items()

dict_items([('e_interaction', <molsberry.core.data.collections.BatchedData object at 0x7f5304c98e90>), ('e_lig_init', <molsberry.core.data.collections.BatchedData object at 0x7f5304c9a410>), ('e_lig_opt', <molsberry.core.data.collections.BatchedData object at 0x7f5304c9a550>)])

In [9]:
{k: v.get_representation_content()[0] for k, v in out.items()}

{'e_interaction': 6.563009999999849,
 'e_lig_init': -27.56031,
 'e_lig_opt': -29.80955}

In [10]:
out['e_lig_init']

In [6]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider
from molsberry.core import Pipeline, InputBlock, OutputBlock

class Divide(Pipeline):
    name = "divider"
    display_name = "Divider Pilpeline"
    def build(self):
        self.add_block(InputBlock(["num1", "num2"]), "input")
        self.add_block(Divider(), "divider")
        self.add_block(OutputBlock(['num_out']), "output")
        self.add_connection("input", "num1", "divider", "num1")
        self.add_connection("input", "num2", "divider", "num2")
        self.add_connection("divider", "num_out", "output", "num_out")

num1 = NumericData(FloatRep(12.1))
num2 = NumericData(FloatRep(12))

pipeline = Divide(base_dir="./example1_out")
out = pipeline.execute(num1 = num1, num2 = num2)


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>            STARTED: Divider Pilpeline             >>
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

[Running Pipe Block: (divider) Numeric divider]
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f53cb21ec90> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>
<class 'molsberry.core.data.generic.NumericData'>
<molsberry.core.data.generic.NumericData object at 0x7f5382988090> <class 'molsberry.core.data.generic.NumericData'>
<class 'molsberry.core.data.generic.FloatRep'>

[Running Pipe Block: (output) Output Block]

<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
<<             ENDED: Divider Pilpeline              <<
<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


In [14]:
num1 = NumericData(FloatRep(12))
num1.get_representation

<bound method Data.get_representation of <molsberry.core.data.generic.NumericData object at 0x7f5382aea690>>

In [8]:
num1 = FloatRep(12.1)
num2 = FloatRep(12)
num1 + num2

24.1

In [9]:
FloatRep(NumericData(out['num_out']))

TypeError: init_rep should be of type Representation.

In [13]:
out['num_out'].get_representation

<bound method BatchedData.get_representation of <molsberry.core.data.collections.BatchedData object at 0x7f538298b4d0>>

In [ ]:
import sys
sys.path.append("..")

from molsberry.core import NumericData, FloatRep
from molsberry.modules.generic.math.blocks import Multiplier, Divider
from molsberry.core import Pipeline, InputBlock, OutputBlock

class Divide(Pipeline):
    name = "divider"
    display_name = "Divider Pilpeline"
    def build(self):
        self.add_block(InputBlock(["num1", "num2", 'num3']), "input")
        self.add_block(Divider(), "divider")
        self.add_block(Multiplier(), 'multiplier')
        self.add_block(OutputBlock(['num_out']), "output")
        self.add_connection("input", "num1", "divider", "num1")
        self.add_connection("input", "num2", "divider", "num2")
        self.add_connection("divider", "num_out", "multiplier", "num_1")
        self._add_connection('input', 'num3', 'multiplier', 'num3')
        self.add_connection('multiplier', 'num_out', 'output', 'num_2')
        

num1 = NumericData(FloatRep(12.1))
num2 = NumericData(FloatRep(12))
num3 = NumericData(FloatRep('14'))

pipeline = Divide(base_dir="./example1_out")
out = pipeline.execute(num1 = num1, num2 = num2, num3 = num3)